In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

        
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={"font.style":"normal",
            "axes.grid":False,
            'figure.figsize':(10.0,10.0)}) 




from sklearn.cluster import KMeans
# Any results you write to the current directory are saved as output.

In [2]:
%sh pip install wordcloud

Requirement already satisfied: wordcloud in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (1.6.0)
Requirement already satisfied: matplotlib in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from wordcloud) (3.0.3)
Requirement already satisfied: numpy>=1.6.1 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from wordcloud) (1.16.2)
Requirement already satisfied: pillow in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from wordcloud) (5.4.1)
Requirement already satisfied: kiwisolver>=1.0.1 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from matplotlib->wordcloud) (1.0.1)
Requirement already satisfied: python-dateutil>=2.1 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from matplotlib->wordcloud) (2.8.0)
Requirement already satisfied: cycler>=0.10 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from matplotlib->wordcloud) (0.10.0)
Requirement already satisfied: pyparsing!=2.0.4,!=2.1.2,!=2.1.6,>=2.0.1 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from matplotlib->wordcloud) (2.3.1)
Requirement already satisfied: setuptools in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from kiwisolver>=1.0.1->matplotlib->wordcloud) (40.8.0)
Requirement already satisfied: six>=1.5 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from python-dateutil>=2.1->matplotlib->wordcloud) (1.12.0)

In [3]:
retail_df = pd.read_csv('/dbfs/FileStore/tables/OnlineRetail.csv',encoding='ISO_8859-1')
retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [4]:
retail_df.CustomerID = retail_df.CustomerID.astype(object)
retail_df.InvoiceDate = pd.to_datetime(retail_df.InvoiceDate,format='%d-%m-%Y %H:%M')

In [5]:
retail_df.shape

Out[9]: (541909, 8)

In [6]:
retail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo 541909 non-null object
StockCode 541909 non-null object
Description 540455 non-null object
Quantity 541909 non-null int64
InvoiceDate 541909 non-null datetime64[ns]
UnitPrice 541909 non-null float64
CustomerID 406829 non-null object
Country 541909 non-null object
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 33.1+ MB

In [7]:
# Droping rows that have missing values

retail_df = retail_df.dropna()

#Dropping negative values
idx = retail_df[retail_df.Quantity < 0].index
retail_df.drop(idx,axis=0,inplace=True)

idx = retail_df[retail_df.UnitPrice < 0].index
retail_df.drop(idx,axis=0,inplace=True)

#Very few outliers, will just drop it for now!
idx = retail_df[retail_df['Quantity'] > 5000].index
retail_df.drop(idx,axis=0,inplace=True)

In [8]:
retail_df.describe()

,Quantity,UnitPrice
count,397921.000000,397921.00000
mean,12.600355,3.11619
std,42.889024,22.09687
min,1.000000,0.00000
25%,2.000000,1.25000
50%,6.000000,1.95000
75%,12.000000,3.75000
max,4800.000000,8142.75000


In [9]:
# DATA VISUALIZATION
# Large number of customers are from UK
plt.figure(figsize=(12,5))
sns.countplot(retail_df['Country'],palette= 'Set3')
plt.xticks(rotation=40,ha='right')
plt.title("Country Distribution")
plt.xlabel('Country')
plt.ylabel('Count');
display()

In [10]:
# More transactions in 2011
plt.figure(figsize=(8,5))
sns.countplot(retail_df['InvoiceDate'].dt.year,palette= 'Set1')
plt.xticks(rotation=40,ha='right')
plt.title("Year Distribution")
plt.xlabel('Year')
plt.ylabel('Count');
display()

In [11]:
#Year ends show more transaction with November being the highest
plt.figure(figsize=(8,5))
sns.countplot(retail_df['InvoiceDate'].dt.month_name(),palette= 'Spectral')
plt.xticks(rotation=40,ha='right')
plt.title("Month Distribution")
plt.xlabel('Month')
plt.ylabel('Count');
display()

In [12]:
#Thursdays have shown more transactions and Fridays the lowest. No transactions on Saturdays !!
plt.figure(figsize=(8,5))
sns.countplot(retail_df['InvoiceDate'].dt.day_name(),palette= 'Set1')
plt.xticks(rotation=40,ha='right')
plt.title("Week Distribution")
plt.xlabel('Week')
plt.ylabel('Count');
display()

In [13]:
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()

In [14]:
#Total_Amount = Total amount spent by the customer Total_Transaction = Total Transactions made by the customer Latest_Transaction = how recently did the customer make the last transaction (in days)

retail_df['Transaction_Amount'] = retail_df['Quantity'] * retail_df['UnitPrice']
df1['Transaction_Amount'] = retail_df.groupby('CustomerID')['Transaction_Amount'].sum()

In [15]:
df2= retail_df.groupby('CustomerID')['InvoiceNo'].count()
df2 = df2.reset_index()
df2.columns = ['CustomerID','Total_Transaction']
df2.head()

,CustomerID,Total_Transaction
0,12347.0,182
1,12348.0,31
2,12349.0,73
3,12350.0,17
4,12352.0,85


In [16]:
df = pd.merge(df1,df2,on='CustomerID',how='inner')

In [17]:
latest_transaction = retail_df['InvoiceDate'].max()

retail_df['Latest_Transaction'] = latest_transaction - retail_df['InvoiceDate']

df3 = retail_df.groupby('CustomerID')['Latest_Transaction'].min()
df3 = df3.reset_index()

df3['Latest_Transaction'] = df3['Latest_Transaction'].dt.days

In [18]:
df = pd.merge(df,df3,on='CustomerID',how='inner')
df.columns = ['CustomerID','Total_Amount','Total_Transaction','Latest_Transaction']
df.head()

,CustomerID,Total_Amount,Total_Transaction,Latest_Transaction
0,12347.0,4310.00,182,1
1,12348.0,1797.24,31,74
2,12349.0,1757.55,73,18
3,12350.0,334.40,17,309
4,12352.0,2506.04,85,35


In [19]:
plt.figure(figsize=(10,5))
sns.boxplot(data = df[['Total_Amount','Total_Transaction','Latest_Transaction']],orient="v", palette="Set2" ,whis=1.5,saturation=1, width=0.7)
display()

In [20]:
df.describe()

,CustomerID,Total_Amount,Total_Transaction,Latest_Transaction
count,4337.000000,4337.000000,4337.000000,4337.000000
mean,15301.089232,1998.098848,91.750288,91.529859
std,1721.422291,8551.826038,228.837406,99.968030
min,12347.000000,2.900000,1.000000,0.000000
25%,13814.000000,307.090000,17.000000,17.000000
50%,15300.000000,673.260000,41.000000,50.000000
75%,16779.000000,1661.330000,100.000000,141.000000
max,18287.000000,280206.020000,7847.000000,373.000000


In [21]:
Q1 = df.Total_Amount.quantile(0.05)
Q3 = df.Total_Amount.quantile(0.95)

IQR = Q3 - Q1
df = df[ (df['Total_Amount']  >= Q1 - 1.5 * IQR) & (df['Total_Amount'] <= Q3 + 1.5 * IQR)]

Q1 = df.Total_Transaction.quantile(0.05)
Q3 = df.Total_Transaction.quantile(0.95)

IQR = Q3 - Q1
df = df[ (df['Total_Transaction']  >= Q1 - 1.5 * IQR) & (df['Total_Transaction'] <= Q3 + 1.5 * IQR)]

Q1 = df.Latest_Transaction.quantile(0.05)
Q3 = df.Latest_Transaction.quantile(0.95)

IQR = Q3 - Q1
df = df[ (df['Latest_Transaction']  >= Q1 - 1.5 * IQR) & (df['Latest_Transaction'] <= Q3 + 1.5 * IQR)]

df

,CustomerID,Total_Amount,Total_Transaction,Latest_Transaction
0,12347.0,4310.00,182,1
1,12348.0,1797.24,31,74
2,12349.0,1757.55,73,18
3,12350.0,334.40,17,309
4,12352.0,2506.04,85,35
5,12353.0,89.00,4,203
6,12354.0,1079.40,58,231
7,12355.0,459.40,13,213
8,12356.0,2811.43,59,22
9,12357.0,6207.67,131,32


In [22]:
plt.figure(figsize=(10,5))
sns.boxplot(data = df[['Total_Amount','Total_Transaction','Latest_Transaction']],orient="v", palette="Set2" ,whis=1.5,saturation=1, width=0.7)
display()

In [23]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled = scaler.fit_transform(df[['Total_Amount','Total_Transaction','Latest_Transaction']])

df_scaled = pd.DataFrame(scaled,columns=['Total_Amount','Total_Transaction','Latest_Transaction'])

df_scaled.head()

,Total_Amount,Total_Transaction,Latest_Transaction
0,1.657337,1.067766,-0.918417
1,0.265403,-0.460499,-0.189896
2,0.243417,-0.035419,-0.748762
3,-0.544932,-0.602193,2.155342
4,0.658040,0.086033,-0.579106


In [24]:
#KMEANS 
#Step 1: Choose the number of clusters k
#Step 2: Select k random points from the data as centroids
#Step 3: Assign all the points to the closest cluster centroid
#Step 4: Recompute the centroids of newly formed clusters
#Step 5: Repeat steps 3 and 4

#Stopping Criteria :
#Centroids of newly formed clusters do not change
#Points remain in the same cluster
#Maximum number of iterations are reached

In [25]:
#Lets fit multiple kmeans
SSE = []

for cluster in range(2,8):
    kmeans = KMeans(n_clusters=cluster,random_state=42)

    kmeans.fit(df_scaled)
    
    centroids = kmeans.cluster_centers_
    pred_clusters = kmeans.predict(df_scaled)
    
    SSE.append(kmeans.inertia_)
    
frame = pd.DataFrame({'Cluster':range(2,8) , 'SSE':SSE})

frame

,Cluster,SSE
0,2,7622.643433
1,3,4427.719029
2,4,3363.937855
3,5,2789.316136
4,6,2391.580656
5,7,2040.103609


In [26]:
plt.figure(figsize=(5,5))
plt.plot(frame['Cluster'],frame['SSE'],marker='o')
plt.title('Custers Vs SSE')
plt.xlabel('No of Clusters')
plt.ylabel('Intertia')
plt.show()
display()
#No of clusters can be anything betwen 3 and 10 as optimum cluster value Let's choose 3!

In [27]:

kmeans = KMeans(n_clusters=3,random_state=42)
kmeans.fit(df_scaled)
pred = kmeans.predict(df_scaled)

In [28]:
#frame = pd.DataFrame(df_scaled)
df['Cluster'] = kmeans.labels_
df['Cluster'].value_counts()

Out[32]: 2 2708
0 1053
1 495
Name: Cluster, dtype: int64

In [29]:
df.head()

,CustomerID,Total_Amount,Total_Transaction,Latest_Transaction,Cluster
0,12347.0,4310.00,182,1,1
1,12348.0,1797.24,31,74,2
2,12349.0,1757.55,73,18,2
3,12350.0,334.40,17,309,0
4,12352.0,2506.04,85,35,2


In [30]:
plt.figure(figsize=(10,5))
sns.boxplot(x = df['Cluster'] ,y = df['Total_Amount'],orient="v", palette="Set2" ,whis=1.5,saturation=1, width=0.7)
plt.title("Clusters Vs Total_Amount")
plt.xlabel("Clusters")
plt.ylabel("Total_Amount")
plt.legend();
display()

#Cluster 1 customers spend more money followed by Cluster 2 customers and lastly Cluster 0

In [31]:
#Cluster 1 customer also buy more frequently followed by Cluster 2 and lastly Cluster 0
plt.figure(figsize=(10,5))
sns.boxplot(x = df['Cluster'] ,y = df['Total_Transaction'],orient="v", palette="Set2" ,whis=1.5,saturation=1, width=0.7)
plt.title("Clusters Vs Total_Transaction")
plt.xlabel("Clusters")
plt.ylabel("Total_Transaction")
plt.legend();
display()

In [32]:
#Cluster 1 customer have made recent transactions, followed by Cluster 2 and lastly Cluster 0
plt.figure(figsize=(10,5))
sns.boxplot(x = df['Cluster'] ,y = df['Latest_Transaction'],orient="v", palette="Set2" ,whis=1.5,saturation=1, width=0.7)
plt.title("Clusters Vs Latest_Transaction")
plt.xlabel("Clusters")
plt.ylabel("Latest_Transaction")
plt.legend();
display()

In [33]:
#So we can conclude that Cluster 1 customers have made more transaction recently and have spent more
plt.scatter(df['Total_Amount'],df['Total_Transaction'],df['Latest_Transaction'],
                     c=kmeans.labels_, cmap='rainbow');display()
